In [2]:
import pandas as pd
df=pd.read_csv(r"C:\Tekworks\Day 6\Employee_Dataset.csv")
df

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active
0,EMP1000,IT,Senior Analyst,60,85000,invalid,01/04/2021,5,3,NaN
1,emp_1,IT,Analyst,NaN,55000,10/06/2020,invalid,5,3,True
2,EMP1002,Sales,NaN,28,85000,NaN,2022-03-01,3,4,NaN
3,EMP1003,it,Analyst,45,35000,invalid,01/04/2021,1,3,NaN
4,NaN,IT,mgr,150,85000,NaN,invalid,12,2,True
...,...,...,...,...,...,...,...,...,...,...
995,EMP1995,Finance,NaN,60,55000,2019-05-10,2022-03-01,1,4,yes
996,EMP1996,IT,Analyst,35,250000,invalid,invalid,-2,5,NaN
997,EMP1997,Sales,Senior Analyst,150,250000,2021/07/15,invalid,3,excellent,yes
998,emp_998,Finance,NaN,60,85000,2019-05-10,NaN,12,NaN,True


In [16]:
#1.Convert joining_date to datetime and count how many rows failed conversion.
df['joining_date_clean'] = pd.to_datetime(df['joining_date'], errors='coerce')
failed_joining_date = df['joining_date_clean'].isna().sum()
failed_joining_date


C:\Users\MAHIPALREDDY\AppData\Local\Temp\ipykernel_11096\2758106654.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['joining_date_clean'] = pd.to_datetime(df['joining_date'], errors='coerce')


np.int64(377)

In [17]:
#2.Clean employee_id and identify how many duplicate employees exist after standardization.
df['employee_id_clean'] = df['employee_id'].astype(str).str.strip().str.upper()
duplicate_employees = df.duplicated('employee_id_clean').sum()
duplicate_employees


np.int64(503)

In [18]:
#3.Standardize department and calculate the average salary per department excluding invalid salaries.
df['department_clean'] = df['department'].str.strip().str.upper()
df['salary_clean'] = pd.to_numeric(df['salary'], errors='coerce')
avg_salary_dept = df[df['salary_clean'] > 0].groupby('department_clean')['salary_clean'].mean()
avg_salary_dept


department_clean
FINANCE    113253.012048
HR         114800.000000
IT         107944.444444
SALES      119000.000000
Name: salary_clean, dtype: float64

In [19]:
#4.Convert age to numeric and find employees with valid salary but invalid age.
df['age_clean'] = pd.to_numeric(df['age'], errors='coerce')
invalid_age_valid_salary = df[
    (df['salary_clean'].notna()) &
    ((df['age_clean'].isna()) | (df['age_clean'] <= 0))
]
invalid_age_valid_salary


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,joining_date_clean,employee_id_clean,department_clean,salary_clean,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean
1,emp_1,IT,Analyst,NaN,55000,10/06/2020,invalid,5,3,True,2020-10-06,EMP_1,IT,55000.0,NaN,3.0,ANALYST,NaT,5.0,True
10,emp_10,Finance,Analyst,-5,35000,NaN,NaN,8,1,no,NaT,EMP_10,FINANCE,35000.0,-5.0,1.0,ANALYST,NaT,8.0,False
23,emp_23,Finance,NaN,-5,85000,invalid,NaN,-2,5,NaN,NaT,EMP_23,FINANCE,85000.0,-5.0,5.0,NaN,NaT,-2.0,NaN
30,EMP1030,HR,Manager,NaN,85000,2019-05-10,invalid,3,3,True,2019-05-10,EMP1030,HR,85000.0,NaN,3.0,MANAGER,NaT,3.0,True
31,NaN,HR,Analyst,NaN,55000,10/06/2020,2022-03-01,1,4,NaN,2020-10-06,NAN,HR,55000.0,NaN,4.0,ANALYST,NaT,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,NaN,sales,mgr,-5,85000,NaN,NaN,-2,NaN,yes,NaT,NAN,SALES,85000.0,-5.0,NaN,MGR,NaT,-2.0,True
982,NaN,HR,NaN,unknown,55000,10/06/2020,NaN,5,NaN,NaN,2020-10-06,NAN,HR,55000.0,NaN,NaN,NaN,NaT,5.0,NaN
990,emp_990,NaN,Senior Analyst,-5,85000,2019-05-10,01/04/2021,12,5,yes,2019-05-10,EMP_990,NaN,85000.0,-5.0,5.0,SENIOR ANALYST,2021-01-04,12.0,True
991,NaN,it,Analyst,NaN,120000,10/06/2020,2022-03-01,-2,4,yes,2020-10-06,NAN,IT,120000.0,NaN,4.0,ANALYST,NaT,-2.0,True


In [20]:
#5.Clean salary and detect outliers using the IQR method.
Q1 = df['salary_clean'].quantile(0.25)
Q3 = df['salary_clean'].quantile(0.75)
IQR = Q3 - Q1
salary_outliers = df[
    (df['salary_clean'] < Q1 - 1.5 * IQR) |
    (df['salary_clean'] > Q3 + 1.5 * IQR)
]
salary_outliers


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,joining_date_clean,employee_id_clean,department_clean,salary_clean,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean
11,NaN,sales,mgr,22,250000,2021/07/15,NaN,NaN,5,yes,2021-07-15,NAN,SALES,250000.0,22.0,5.0,MGR,NaT,NaN,True
16,NaN,IT,NaN,60,250000,NaN,invalid,12,poor,no,NaT,NAN,IT,250000.0,60.0,NaN,NaN,NaT,12.0,False
25,NaN,Sales,NaN,28,250000,invalid,NaN,8,NaN,True,NaT,NAN,SALES,250000.0,28.0,NaN,NaN,NaT,8.0,True
34,NaN,sales,Analyst,45,250000,NaN,2022-03-01,-2,5,False,NaT,NAN,SALES,250000.0,45.0,5.0,ANALYST,NaT,-2.0,False
41,NaN,NaN,ANALYST,60,250000,2021/07/15,NaN,3,4,yes,2021-07-15,NAN,NaN,250000.0,60.0,4.0,ANALYST,NaT,3.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,NaN,it,ANALYST,60,250000,2019-05-10,01/04/2021,NaN,3,no,2019-05-10,NAN,IT,250000.0,60.0,3.0,ANALYST,2021-01-04,NaN,False
969,EMP1969,it,NaN,150,250000,2019-05-10,2022-03-01,NaN,NaN,NaN,2019-05-10,EMP1969,IT,250000.0,150.0,NaN,NaN,NaT,NaN,NaN
976,EMP1976,HR,Analyst,unknown,250000,2019-05-10,NaN,1,poor,False,2019-05-10,EMP1976,HR,250000.0,NaN,NaN,ANALYST,NaT,1.0,False
996,EMP1996,IT,Analyst,35,250000,invalid,invalid,-2,5,NaN,NaT,EMP1996,IT,250000.0,35.0,5.0,ANALYST,NaT,-2.0,NaN


In [21]:
#6.Convert performance_rating into numeric and calculate the median rating per designation.
rating_map = {'POOR':1, 'AVERAGE':2, 'GOOD':3, 'VERY GOOD':4, 'EXCELLENT':5}
df['performance_clean'] = (
    df['performance_rating']
    .replace(rating_map)
    .apply(pd.to_numeric, errors='coerce')
)
df['designation_clean'] = df['designation'].str.strip().str.upper()
median_rating_designation = df.groupby('designation_clean')['performance_clean'].median()
median_rating_designation


designation_clean
ANALYST           3.0
MANAGER           3.0
MGR               3.0
SENIOR ANALYST    3.0
Name: performance_clean, dtype: float64

In [22]:
#7.Identify employees whose last_promotion_date is earlier than their joining_date.
df['last_promotion_date_clean'] = pd.to_datetime(df['last_promotion_date'], errors='coerce')
invalid_promotion = df[
    df['last_promotion_date_clean'] < df['joining_date_clean']
]
invalid_promotion


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,joining_date_clean,employee_id_clean,department_clean,salary_clean,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean
24,EMP1024,Finance,mgr,35,85000,2021/07/15,01/04/2021,8,3,True,2021-07-15,EMP1024,FINANCE,85000.0,35.0,3.0,MGR,2021-01-04,8.0,True
40,emp_40,NaN,NaN,60,85000,2021/07/15,01/04/2021,12,5,True,2021-07-15,EMP_40,NaN,85000.0,60.0,5.0,NaN,2021-01-04,12.0,True
44,emp_44,sales,mgr,45,₹75000,2021/07/15,01/04/2021,8,2,True,2021-07-15,EMP_44,SALES,NaN,45.0,2.0,MGR,2021-01-04,8.0,True
47,NaN,Finance,Analyst,unknown,250000,2021/07/15,01/04/2021,3,excellent,True,2021-07-15,NAN,FINANCE,250000.0,NaN,NaN,ANALYST,2021-01-04,3.0,True
71,emp_71,Sales,Manager,60,250000,2021/07/15,01/04/2021,12,excellent,False,2021-07-15,EMP_71,SALES,250000.0,60.0,NaN,MANAGER,2021-01-04,12.0,False
82,emp_82,sales,Senior Analyst,150,NaN,2021/07/15,01/04/2021,-2,1,yes,2021-07-15,EMP_82,SALES,NaN,150.0,1.0,SENIOR ANALYST,2021-01-04,-2.0,True
85,NaN,Sales,NaN,60,55000,2021/07/15,01/04/2021,NaN,2,yes,2021-07-15,NAN,SALES,55000.0,60.0,2.0,NaN,2021-01-04,NaN,True
124,NaN,sales,Analyst,unknown,250000,2021/07/15,01/04/2021,1,5,no,2021-07-15,NAN,SALES,250000.0,NaN,5.0,ANALYST,2021-01-04,1.0,False
138,emp_138,Finance,mgr,45,250000,2021/07/15,01/04/2021,NaN,3,False,2021-07-15,EMP_138,FINANCE,250000.0,45.0,3.0,MGR,2021-01-04,NaN,False
169,emp_169,it,mgr,60,120000,2021/07/15,01/04/2021,NaN,2,NaN,2021-07-15,EMP_169,IT,120000.0,60.0,2.0,MGR,2021-01-04,NaN,NaN


In [23]:
#8.Clean experience_years and find mismatches where experience exceeds employee age.
df['experience_clean'] = pd.to_numeric(df['experience_years'], errors='coerce')
experience_age_mismatch = df[
    df['experience_clean'] > df['age_clean']
]
experience_age_mismatch


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,joining_date_clean,employee_id_clean,department_clean,salary_clean,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean
10,emp_10,Finance,Analyst,-5,35000,NaN,NaN,8,1,no,NaT,EMP_10,FINANCE,35000.0,-5.0,1.0,ANALYST,NaT,8.0,False
17,emp_17,sales,Senior Analyst,-5,NaN,NaN,01/04/2021,3,4,False,NaT,EMP_17,SALES,NaN,-5.0,4.0,SENIOR ANALYST,2021-01-04,3.0,False
23,emp_23,Finance,NaN,-5,85000,invalid,NaN,-2,5,NaN,NaT,EMP_23,FINANCE,85000.0,-5.0,5.0,NaN,NaT,-2.0,NaN
43,NaN,it,Manager,-5,120000,invalid,NaN,1,excellent,NaN,NaT,NAN,IT,120000.0,-5.0,NaN,MANAGER,NaT,1.0,NaN
49,NaN,it,mgr,-5,55000,2021/07/15,invalid,5,3,False,2021-07-15,NAN,IT,55000.0,-5.0,3.0,MGR,NaT,5.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,NaN,sales,mgr,-5,NaN,10/06/2020,NaN,12,poor,no,2020-10-06,NAN,SALES,NaN,-5.0,NaN,MGR,NaT,12.0,False
981,NaN,sales,mgr,-5,85000,NaN,NaN,-2,NaN,yes,NaT,NAN,SALES,85000.0,-5.0,NaN,MGR,NaT,-2.0,True
985,emp_985,sales,mgr,-5,NaN,invalid,invalid,-2,1,NaN,NaT,EMP_985,SALES,NaN,-5.0,1.0,MGR,NaT,-2.0,NaN
990,emp_990,NaN,Senior Analyst,-5,85000,2019-05-10,01/04/2021,12,5,yes,2019-05-10,EMP_990,NaN,85000.0,-5.0,5.0,SENIOR ANALYST,2021-01-04,12.0,True


In [27]:
#9.Standardize designation and count how many active employees are in each  designation.
df['designation_clean'] = (
    df['designation']
    .astype(str)
    .str.strip()
    .str.upper()
)


In [28]:
df['is_active_clean'] = (
    df['is_active']
    .astype(str)
    .str.strip()
    .str.lower()
    .map({'true': True, 'false': False, 'yes': True, 'no': False})
)


In [29]:
active_employee_count = (
    df[df['is_active_clean'] == True]
    .groupby('designation_clean')
    .size()
)
active_employee_count


designation_clean
ANALYST           122
MANAGER            68
MGR                82
NAN                62
SENIOR ANALYST     88
dtype: int64

In [35]:
#10.Convert is_active to boolean and find inactive employees with recent promotions.
recent_inactive = df[
    (df['is_active_clean']) &
    (df['last_promotion_date_clean'] >= pd.Timestamp.now() - pd.DateOffset(years=2))
]
recent_inactive


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,joining_date_clean,employee_id_clean,department_clean,salary_clean,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean


In [ ]:
#11.Calculate employee tenure in years and find those with tenure above the 90th percentile.
df['tenure_years'] = (pd.Timestamp.now() - df['joining_date_clean']).dt.days / 365
threshold_90 = df['tenure_years'].quantile(0.9)
long_tenure_employees = df[df['tenure_years'] > threshold_90]
long_tenure_employees


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,employee_id_clean,department_clean,salary_clean,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean,tenure_years


In [38]:
#12.Identify departments where more than 25% of salary values are missing or invalid.
invalid_salary_pct = (
    df['salary_clean'].isna()
    .groupby(df['department_clean'])
    .mean()
)
high_invalid_salary_depts = invalid_salary_pct[invalid_salary_pct > 0.25]
high_invalid_salary_depts


department_clean
FINANCE    0.366412
HR         0.321267
IT         0.340659
SALES      0.398340
Name: salary_clean, dtype: float64

In [39]:
#13.Create a flag for employees with high performance (≥4) but below-median salary.
median_salary = df['salary_clean'].median()
df['high_perf_low_salary_flag'] = (
    (df['performance_clean'] >= 4) &
    (df['salary_clean'] < median_salary)
)
df[['employee_id', 'high_perf_low_salary_flag']]


,employee_id,high_perf_low_salary_flag
0,EMP1000,False
1,emp_1,False
2,EMP1002,False
3,EMP1003,False
4,NaN,False
...,...,...
995,EMP1995,True
996,EMP1996,False
997,EMP1997,False
998,emp_998,False


In [40]:
#14.Detect employees with no promotion date but more than 5 years of experience.
no_promo_high_exp = df[
    (df['last_promotion_date_clean'].isna()) &
    (df['experience_clean'] > 5)
]
no_promo_high_exp


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,department_clean,salary_clean,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean,tenure_years,high_perf_low_salary_flag
4,NaN,IT,mgr,150,85000,NaN,invalid,12,2,True,...,IT,85000.0,150.0,2.0,MGR,NaT,12.0,True,NaN,False
5,NaN,IT,mgr,NaN,₹75000,2019-05-10,invalid,8,3,no,...,IT,NaN,NaN,3.0,MGR,NaT,8.0,False,6.756164,False
6,NaN,HR,NaN,35,120000,10/06/2020,NaN,12,poor,NaN,...,HR,120000.0,35.0,NaN,NAN,NaT,12.0,NaN,5.345205,False
10,emp_10,Finance,Analyst,-5,35000,NaN,NaN,8,1,no,...,FINANCE,35000.0,-5.0,1.0,ANALYST,NaT,8.0,False,NaN,False
12,emp_12,Sales,Analyst,60,₹75000,NaN,NaN,8,NaN,NaN,...,SALES,NaN,60.0,NaN,ANALYST,NaT,8.0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,emp_977,IT,ANALYST,28,NaN,2019-05-10,invalid,8,4,NaN,...,IT,NaN,28.0,4.0,ANALYST,NaT,8.0,NaN,6.756164,False
978,EMP1978,HR,Senior Analyst,28,₹75000,10/06/2020,invalid,8,1,True,...,HR,NaN,28.0,1.0,SENIOR ANALYST,NaT,8.0,True,5.345205,False
993,EMP1993,sales,NaN,35,120000,10/06/2020,2022-03-01,12,NaN,no,...,SALES,120000.0,35.0,NaN,NAN,NaT,12.0,False,5.345205,False
994,EMP1994,Sales,NaN,-5,85000,2019-05-10,2022-03-01,12,1,True,...,SALES,85000.0,-5.0,1.0,NAN,NaT,12.0,True,6.756164,False


In [41]:
#15.Build a validation rule to flag rows violating at least two business constraints.
df['violation_count'] = (
    (df['salary_clean'].isna()).astype(int) +
    (df['age_clean'].isna()).astype(int) +
    (df['experience_clean'].isna()).astype(int) +
    (df['joining_date_clean'].isna()).astype(int)
)
df['invalid_row_flag'] = df['violation_count'] >= 2
df[df['invalid_row_flag']]


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean,tenure_years,high_perf_low_salary_flag,violation_count,invalid_row_flag
5,NaN,IT,mgr,NaN,₹75000,2019-05-10,invalid,8,3,no,...,NaN,3.0,MGR,NaT,8.0,False,6.756164,False,2,True
12,emp_12,Sales,Analyst,60,₹75000,NaN,NaN,8,NaN,NaN,...,60.0,NaN,ANALYST,NaT,8.0,NaN,NaN,False,2,True
17,emp_17,sales,Senior Analyst,-5,NaN,NaN,01/04/2021,3,4,False,...,-5.0,4.0,SENIOR ANALYST,2021-01-04,3.0,False,NaN,False,2,True
18,emp_18,Finance,Senior Analyst,NaN,NaN,invalid,NaN,ten,2,yes,...,NaN,2.0,SENIOR ANALYST,NaT,NaN,True,NaN,False,4,True
19,NaN,IT,mgr,22,120000,NaN,2022-03-01,ten,4,yes,...,22.0,4.0,MGR,NaT,NaN,True,NaN,False,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,NaN,IT,NaN,60,NaN,NaN,01/04/2021,5,poor,False,...,60.0,NaN,NAN,2021-01-04,5.0,False,NaN,False,2,True
983,EMP1983,HR,mgr,150,NaN,invalid,01/04/2021,1,1,False,...,150.0,1.0,MGR,2021-01-04,1.0,False,NaN,False,2,True
984,NaN,sales,Manager,35,₹75000,invalid,2022-03-01,5,1,no,...,35.0,1.0,MANAGER,NaT,5.0,False,NaN,False,2,True
985,emp_985,sales,mgr,-5,NaN,invalid,invalid,-2,1,NaN,...,-5.0,1.0,MGR,NaT,-2.0,NaN,NaN,False,2,True


In [42]:
#1.After cleaning dates, find employees whose promotion gap (promotion date − joining date) is less than 1 year.
promo_gap = (df['last_promotion_date_clean'] - df['joining_date_clean']).dt.days / 365
df[promo_gap < 1]


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean,tenure_years,high_perf_low_salary_flag,violation_count,invalid_row_flag
9,EMP1009,it,NaN,22,120000,10/06/2020,01/04/2021,NaN,4,NaN,...,22.0,4.0,NAN,2021-01-04,NaN,NaN,5.345205,False,1,False
24,EMP1024,Finance,mgr,35,85000,2021/07/15,01/04/2021,8,3,True,...,35.0,3.0,MGR,2021-01-04,8.0,True,4.572603,False,0,False
40,emp_40,NaN,NaN,60,85000,2021/07/15,01/04/2021,12,5,True,...,60.0,5.0,NAN,2021-01-04,12.0,True,4.572603,False,0,False
44,emp_44,sales,mgr,45,₹75000,2021/07/15,01/04/2021,8,2,True,...,45.0,2.0,MGR,2021-01-04,8.0,True,4.572603,False,1,False
47,NaN,Finance,Analyst,unknown,250000,2021/07/15,01/04/2021,3,excellent,True,...,NaN,NaN,ANALYST,2021-01-04,3.0,True,4.572603,False,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,EMP1943,Finance,mgr,45,35000,10/06/2020,01/04/2021,-2,NaN,NaN,...,45.0,NaN,MGR,2021-01-04,-2.0,NaN,5.345205,False,0,False
945,NaN,HR,Manager,-5,35000,10/06/2020,01/04/2021,3,NaN,True,...,-5.0,NaN,MANAGER,2021-01-04,3.0,True,5.345205,False,0,False
966,NaN,it,Analyst,45,NaN,10/06/2020,01/04/2021,1,NaN,no,...,45.0,NaN,ANALYST,2021-01-04,1.0,False,5.345205,False,1,False
974,emp_974,NaN,mgr,unknown,85000,2021/07/15,01/04/2021,3,4,False,...,NaN,4.0,MGR,2021-01-04,3.0,False,4.572603,False,1,False


In [43]:
#2.Identify employees whose salary is above the department average but performance rating is below the department median.
dept_avg_salary = df.groupby('department_clean')['salary_clean'].transform('mean')
dept_median_perf = df.groupby('department_clean')['performance_clean'].transform('median')
df[
    (df['salary_clean'] > dept_avg_salary) &
    (df['performance_clean'] < dept_median_perf)
]


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,age_clean,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean,tenure_years,high_perf_low_salary_flag,violation_count,invalid_row_flag
45,NaN,Finance,NaN,NaN,120000,NaN,01/04/2021,1,1,yes,...,NaN,1.0,NAN,2021-01-04,1.0,True,NaN,False,2,True
115,emp_115,sales,ANALYST,28,250000,2021/07/15,invalid,-2,2,no,...,28.0,2.0,ANALYST,NaT,-2.0,False,4.572603,False,0,False
127,NaN,Finance,NaN,150,120000,invalid,2022-03-01,3,1,True,...,150.0,1.0,NAN,NaT,3.0,True,NaN,False,1,False
157,NaN,Finance,ANALYST,22,250000,2019-05-10,2022-03-01,3,1,False,...,22.0,1.0,ANALYST,NaT,3.0,False,6.756164,False,0,False
169,emp_169,it,mgr,60,120000,2021/07/15,01/04/2021,NaN,2,NaN,...,60.0,2.0,MGR,2021-01-04,NaN,NaN,4.572603,False,1,False
173,emp_173,Finance,Manager,45,120000,10/06/2020,invalid,8,1,False,...,45.0,1.0,MANAGER,NaT,8.0,False,5.345205,False,0,False
217,NaN,it,NaN,28,250000,2021/07/15,2022-03-01,NaN,2,yes,...,28.0,2.0,NAN,NaT,NaN,True,4.572603,False,1,False
223,NaN,HR,Manager,NaN,120000,2019-05-10,NaN,-2,1,True,...,NaN,1.0,MANAGER,NaT,-2.0,True,6.756164,False,1,False
270,NaN,IT,ANALYST,NaN,120000,10/06/2020,invalid,-2,2,NaN,...,NaN,2.0,ANALYST,NaT,-2.0,NaN,5.345205,False,1,False
273,NaN,Finance,ANALYST,NaN,120000,10/06/2020,NaN,12,2,yes,...,NaN,2.0,ANALYST,NaT,12.0,True,5.345205,False,1,False


In [44]:
#3.Create a column that categorizes employees into age bands (Young, Mid, Senior) and count employees per band per department.
df['age_band'] = pd.cut(
    df['age_clean'],
    bins=[0,30,50,100],
    labels=['Young','Mid','Senior']
)
df.groupby(['department_clean','age_band']).size()


C:\Users\MAHIPALREDDY\AppData\Local\Temp\ipykernel_11096\4180047844.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['department_clean','age_band']).size()


department_clean  age_band
FINANCE           Young       33
                  Mid         31
                  Senior      13
HR                Young       54
                  Mid         52
                  Senior      22
IT                Young       53
                  Mid         78
                  Senior      37
SALES             Young       48
                  Mid         57
                  Senior      25
dtype: int64

In [45]:
#4.Find departments where the average experience is higher than the company-wide average experience.
company_avg_exp = df['experience_clean'].mean()
dept_above_avg_exp = df.groupby('department_clean')['experience_clean'].mean()
dept_above_avg_exp[dept_above_avg_exp > company_avg_exp]


department_clean
FINANCE    4.826923
SALES      4.709677
Name: experience_clean, dtype: float64

In [47]:
#5.Detect employees marked as inactive but having a valid promotion date within the last 2 years.
df[
    (df['is_active_clean']) &
    (df['last_promotion_date_clean'] >= pd.Timestamp.now() - pd.DateOffset(years=2))
]


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,performance_clean,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean,tenure_years,high_perf_low_salary_flag,violation_count,invalid_row_flag,age_band


In [48]:
#6.Identify designations where more than 20% of employees have missing or invalid experience values.
invalid_exp_ratio = df['experience_clean'].isna().groupby(df['designation_clean']).mean()
invalid_exp_ratio[invalid_exp_ratio > 0.20]


designation_clean
MGR               0.264045
SENIOR ANALYST    0.202128
Name: experience_clean, dtype: float64

In [49]:
#7.Calculate the ratio of salary to experience years and detect extreme values using the 95th percentile.
df['salary_exp_ratio'] = df['salary_clean'] / df['experience_clean']
threshold_95 = df['salary_exp_ratio'].quantile(0.95)
df[df['salary_exp_ratio'] > threshold_95]


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean,tenure_years,high_perf_low_salary_flag,violation_count,invalid_row_flag,age_band,salary_exp_ratio
124,NaN,sales,Analyst,unknown,250000,2021/07/15,01/04/2021,1,5,no,...,ANALYST,2021-01-04,1.0,False,4.572603,False,1,False,NaN,250000.0
133,NaN,it,NaN,-5,250000,2021/07/15,2022-03-01,1,5,True,...,NAN,NaT,1.0,True,4.572603,False,0,False,NaN,250000.0
176,NaN,it,Analyst,22,250000,invalid,NaN,1,NaN,False,...,ANALYST,NaT,1.0,False,NaN,False,1,False,Young,250000.0
224,NaN,Finance,Manager,60,250000,2021/07/15,invalid,1,3,NaN,...,MANAGER,NaT,1.0,NaN,4.572603,False,0,False,Senior,250000.0
262,NaN,IT,Manager,-5,250000,NaN,invalid,1,5,NaN,...,MANAGER,NaT,1.0,NaN,NaN,False,1,False,NaN,250000.0
399,NaN,Finance,ANALYST,60,250000,invalid,NaN,1,3,True,...,ANALYST,NaT,1.0,True,NaN,False,1,False,Senior,250000.0
401,EMP1401,Finance,Analyst,22,250000,NaN,invalid,1,1,NaN,...,ANALYST,NaT,1.0,NaN,NaN,False,1,False,Young,250000.0
470,EMP1470,HR,mgr,35,250000,2021/07/15,2022-03-01,1,excellent,True,...,MGR,NaT,1.0,True,4.572603,False,0,False,Mid,250000.0
525,emp_525,sales,Manager,-5,250000,10/06/2020,2022-03-01,1,1,False,...,MANAGER,NaT,1.0,False,5.345205,False,0,False,NaN,250000.0
529,emp_529,NaN,NaN,28,250000,10/06/2020,invalid,1,5,no,...,NAN,NaT,1.0,False,5.345205,False,0,False,Young,250000.0


In [50]:
#8.Find employees whose age is valid but inconsistent with experience (experience > age − 18).
df[df['experience_clean'] > (df['age_clean'] - 18)]


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,designation_clean,last_promotion_date_clean,experience_clean,is_active_clean,tenure_years,high_perf_low_salary_flag,violation_count,invalid_row_flag,age_band,salary_exp_ratio
10,emp_10,Finance,Analyst,-5,35000,NaN,NaN,8,1,no,...,ANALYST,NaT,8.0,False,NaN,False,1,False,NaN,4375.000000
17,emp_17,sales,Senior Analyst,-5,NaN,NaN,01/04/2021,3,4,False,...,SENIOR ANALYST,2021-01-04,3.0,False,NaN,False,2,True,NaN,NaN
23,emp_23,Finance,NaN,-5,85000,invalid,NaN,-2,5,NaN,...,NAN,NaT,-2.0,NaN,NaN,False,1,False,NaN,-42500.000000
43,NaN,it,Manager,-5,120000,invalid,NaN,1,excellent,NaN,...,MANAGER,NaT,1.0,NaN,NaN,False,1,False,NaN,120000.000000
49,NaN,it,mgr,-5,55000,2021/07/15,invalid,5,3,False,...,MGR,NaT,5.0,False,4.572603,False,0,False,NaN,11000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,emp_968,IT,Manager,22,120000,2021/07/15,2022-03-01,5,poor,True,...,MANAGER,NaT,5.0,True,4.572603,False,0,False,Young,24000.000000
981,NaN,sales,mgr,-5,85000,NaN,NaN,-2,NaN,yes,...,MGR,NaT,-2.0,True,NaN,False,1,False,NaN,-42500.000000
985,emp_985,sales,mgr,-5,NaN,invalid,invalid,-2,1,NaN,...,MGR,NaT,-2.0,NaN,NaN,False,2,True,NaN,NaN
990,emp_990,NaN,Senior Analyst,-5,85000,2019-05-10,01/04/2021,12,5,yes,...,SENIOR ANALYST,2021-01-04,12.0,True,6.756164,False,0,False,NaN,7083.333333


In [52]:
#9.Compute department-wise attrition rate assuming inactive employees represent attrition.
attrition_rate = (
    (df['is_active_clean'])
    .groupby(df['department_clean'])
    .mean()
)
attrition_rate


department_clean
FINANCE    0.574257
HR         0.553073
IT         0.530516
SALES      0.482412
Name: is_active_clean, dtype: object

In [54]:
#10.Create a final quality score per row based on number of valid fields and flag rows below a quality threshold.
valid_fields = df[
    ['age_clean','salary_clean','experience_clean','joining_date_clean','performance_clean']
].notna().sum(axis=1)
df['quality_score'] = valid_fields
df['low_quality_flag'] = df['quality_score'] < 3
df[['employee_id','quality_score','low_quality_flag']]


,employee_id,quality_score,low_quality_flag
0,EMP1000,4,False
1,emp_1,4,False
2,EMP1002,4,False
3,EMP1003,4,False
4,NaN,4,False
...,...,...,...
995,EMP1995,5,False
996,EMP1996,4,False
997,EMP1997,4,False
998,emp_998,4,False
